# Interpreter

## Numerical Eval

In [10]:
from enum import Enum
from abc import abstractmethod

class Element:
    @abstractmethod
    def eval(self):
        pass

class Integer(Element):
    def __init__(self, i):
        self.value = i

    def eval(self):
        return self.value

    def __str__(self):
        return f"{self.value}"

class BinaryOp(Element):
    class Op(Enum):
        add = 1,
        sub = 2

    def __init__(self, t:Op=None, l: Element=None, r: Element=None):
        self.op = t
        self.lvalue = l
        self.rvalue = r

    def eval(self):
        if self.op == self.Op.add:
            return self.lvalue.eval() + self.rvalue.eval()
        else:
            return self.lvalue.eval() - self.rvalue.eval()

    def __str__(self):
        op = "+" if self.op == self.Op.add else "-"
        return f"{self.lvalue}{op}{self.rvalue}"


class UnaryOp(Element):
    class Op(Enum):
        increment = 1,
        decrement = 2,
        ref = 3
    
    def __init__(self, t:Op=None, r:Element=None):
        self.op = t
        self.rvalue = r

    def eval(self):
        if self.op == self.Op.increment:
            self.rvalue += 1
        elif self.op == self.Op.decrement:
            self.rvalue -= 1 
        elif self.op == self.Op.ref:
            return id(self.rvalue)
        else:
            raise NotImplementedError
            
    def __str__(self):
        op = "++" if self.op == self.Op.increment else "--"
        return f"{op}{self.rvalue}"

In [11]:
expr = BinaryOp(BinaryOp.Op.add, Integer(2), Integer(3))
print(expr.eval())
expr = UnaryOp(UnaryOp.Op.ref, Integer(4))
print(expr.eval())

5
139778021623440


## Parser

In [12]:
# a parser
class Parser:
    def __init__(self, string):
        self.string = string
        self.expr = None
          
    def __call__(self):
        has_lvalue = False
        is_uni = False
        expr = None
        op = None
        lvalue = None
        rvalue = None
        for c in self.string:
            if c == "+":
                op = BinaryOp.Op.add
            elif c == "-":
                op = BinaryOp.Op.sub
            elif c == "++":
                op = UnaryOp.Op.increment
                has_lvalue = True
                is_uni = True
            elif c == "--":
                op = UnaryOp.Op.decrement
                has_lvalue = True
                is_uni = True
            elif c.isnumeric() and not has_lvalue:
                lvalue = Integer(int(c))
                has_lvalue = True
            elif c.isnumeric():
                rvalue = Integer(int(c))

        if is_uni:
            self.expr = UnaryOp(op, rvalue)
        else:
            self.expr = BinaryOp(op, lvalue, rvalue)
        return self.expr

    def __str__(self):
        return f"{self.expr}"

In [13]:
parsed = Parser("3+4")()
res = parsed.eval()
print(f"{parsed} = {res}")

3+4 = 7


https://morepypy.blogspot.com/2011/04/tutorial-writing-interpreter-with-pypy.html